<a href="https://colab.research.google.com/github/squadOito/soulcodead2/blob/joseaureliok%2Fnotebook/notebooks/notebook_geracao_distribuida_bruto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Projeto Final**
Escola: SoulCode Academy

Curso: Bootcamp Analista de Dados - Martech - AD2

**Alunos: Nomes, Nomes**

Professores: Douglas Ribeiro e Franciane Rodrigues

## **Pergunta de negócio**: Análise exploratória de dados com a base de dados do ___ e indique possíveis oportunidades e/ou ameaças para o empreendimento.

---



# ETL
Extração, transformação, carregamento dos dados



## Preparação de Ambiente
Instalações e importações das bibliotecas necessárias para o processo de ETL.

In [1]:
# instalaçao de biblioteca para interagir com google cloud
!pip install gcsfs -q

In [2]:
# instalaçao de biblioteca para validar dados com pandera
!pip install pandera -q

In [3]:
# Importando Bibliotecas
import os
import pandas as pd
import numpy as np
import pandera as pa
import re

from google.cloud import storage
from google.colab import drive
from google.colab import auth
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

In [4]:
# Ignorando alguns alertas desnecessários
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Configuração da quantidade de colunas para aparecer em um DataFrame
pd.set_option('display.max_columns',100)

### Compartilhamento Chave GDrive

In [6]:
# Cria compartilhamento com Google Drive
drive.mount('/content/drive', force_remount=True)

# Arquivo a ser acessado na pasta compartilhada
target = 'projeto-final-ad2-e8-ae566c3a2c2b.json'

# Caminho completo da pasta compartilhada
folder = '/content/drive/MyDrive/Classroom/AD2 - Analista de Dados/ProjetoFinal'

# Acesso ao arquivo no colab
serviceAccount = os.path.join(folder, target)

Mounted at /content/drive


## Extração
A primeira etapa da ETL é a extração dos dados de sua fonte original. Dependendo do tipo de dados e da fonte, você pode precisar de diferentes ferramentas e técnicas para extrair os dados.

In [7]:
# Conexão com a conta do Google Cloud
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = serviceAccount

In [8]:
# Conexão com a bucket do Google Cloud
client = storage.Client()
bucket = client.get_bucket('projeto-final-ad2-e8') # nome do bucket

In [9]:
# Caminhodo arquivo
path = "gs://projeto-final-ad2-e8/dados/pre-analise/geracao_distribuida_pre.csv" # gsutil

*Read* Pandas

In [10]:
# Leitura dataframe pandas
read = pd.read_csv(path,
                         sep= ',',
                         encoding = 'utf-8')

In [11]:
# exibindo primeiras linhas
read.head(10)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,2023-07-31,07/2023,8.336783e+12,CELESC-DIS,CELESC DISTRIBUI�ÃO S.A,5,Poder Público,11,B3,42.0,SC,4204.0,Sul,4210852.0,Mirim Doce,00000000,PJ,95952289000155,MIRIM DOCE CAMARA DE VEREADORES,GD.SC.000.188.684,2020-01-20,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,500,NaN,NaN,NaN,NaN,NaN
1,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,01109010,PJ,10882594001722,"INSTITUTO FEDERAL DE EDUCA�ÃO, CIÊNCIA E TECN.",GD.SP.000.027.612,2017-07-12,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6996,NaN,NaN,NaN,NaN,NaN
2,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,04115***,PF,***.015.418-**,***,GD.SP.000.081.118,2019-05-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
3,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3510609.0,Carapicuíba,06364***,PF,***.517.398-**,***,GD.SP.000.210.094,2020-03-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
4,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3505708.0,Barueri,06462080,PJ,46523015000135,MUNICIPIO DE BARUERI,GD.SP.000.863.525,2021-04-02,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,2300,-2351.0,-4688.0,NaN,NaN,NaN
5,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,11,B3,35.0,SP,3515.0,Sudeste,3518800.0,Guarulhos,07160640,PJ,46319000000150,MUNICIPIO DE GUARULHOS,GD.SP.000.251.356,2020-06-09,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,284,-2346.0,-4650.0,NaN,NaN,NaN
6,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,11,B3,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,08265060,PJ,74118514000182,SAO PAULO SECRETARIA VERDE MEIO AMB,GD.SP.000.049.685,2018-12-11,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
7,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3552502.0,Suzano,08683000,PJ,43776517000180,CIA DE SANEAMENTO BASICO DO ESTADO DE S,GD.SP.001.908.917,2023-06-07,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Minigeracao,200000,74.0,-20.0,NaN,NaN,NaN
8,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3530607.0,Mogi das Cruzes,08750770,PJ,46523270000188,MUNICIPIO DE MOGI DAS CRUZES,GD.SP.000.351.911,2020-10-19,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6000,-2300.0,-4700.0,NaN,NaN,NaN
9,2023-07-31,07/2023,2.302100e+12,EDP SP,EDP SÃO PAULO DISTRIBUI�ÃO DE ENERGIA S.A.,5,Poder Público,9,B1,35.0,SP,3513.0,Sudeste,3550704.0,São Sebastião,11619***,PF,***.090.248-**,***,GD.SP.000.314.370,2020-10-05,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,200,NaN,NaN,NaN,NaN,NaN


In [12]:
# renomeação do dataframe lido
df = read

## Pré-análise

In [13]:
# Visualização geral
display(df)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
0,2023-07-31,07/2023,8.336783e+12,CELESC-DIS,CELESC DISTRIBUI�ÃO S.A,5,Poder Público,11,B3,42.0,SC,4204.0,Sul,4210852.0,Mirim Doce,00000000,PJ,95952289000155,MIRIM DOCE CAMARA DE VEREADORES,GD.SC.000.188.684,2020-01-20,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,500,NaN,NaN,NaN,NaN,NaN
1,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,6,Serviço Público,7,A4,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,01109010,PJ,10882594001722,"INSTITUTO FEDERAL DE EDUCA�ÃO, CIÊNCIA E TECN.",GD.SP.000.027.612,2017-07-12,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,6996,NaN,NaN,NaN,NaN,NaN
2,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3550308.0,São Paulo,04115***,PF,***.015.418-**,***,GD.SP.000.081.118,2019-05-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
3,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,9,B1,35.0,SP,3515.0,Sudeste,3510609.0,Carapicuíba,06364***,PF,***.517.398-**,***,GD.SP.000.210.094,2020-03-30,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
4,2023-07-31,07/2023,6.169523e+13,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,Poder Público,7,A4,35.0,SP,3515.0,Sudeste,3505708.0,Barueri,06462080,PJ,46523015000135,MUNICIPIO DE BARUERI,GD.SP.000.863.525,2021-04-02,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,2300,-2351.0,-4688.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053216,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,9,B1,24.0,RN,2404.0,Nordeste,2403251.0,Parnamirim,59152***,PF,***.368.374-**,***,GD.RN.000.549.804,2021-07-31,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,737,NaN,NaN,NaN,NaN,NaN
2053217,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,9,B1,24.0,RN,2402.0,Nordeste,2402006.0,Caicó,59300***,PF,***.405.104-**,***,GD.RN.000.863.246,2022-02-25,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
2053218,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,Ilumina�ão pública,10,B2,24.0,RN,2402.0,Nordeste,2407203.0,Macau,59500***,PF,***.771.634-**,***,GD.RN.000.349.903,2020-11-06,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,600,NaN,NaN,NaN,NaN,NaN
2053219,2023-07-31,07/2023,7.047251e+12,ENEL CE,COMPANHIA ENERGÉTICA DO CEARÁ,7,Ilumina�ão pública,11,B3,23.0,CE,2303.0,Nordeste,2304400.0,Fortaleza,60878085,PJ,23552466000103,RENOVADORA DE P SAO LUIS LTDA,GD.CE.000.166.397,2019-12-31,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,4460,NaN,NaN,NaN,NaN,NaN


In [14]:
# Visualização de forma aleatoria
df.sample(20)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
432402,2023-07-31,07/2023,2.508603e+13,ETO,ENERGISA TOCANTINS DISTRIBUIDORA DE ENERGIA S.A.,1,Residencial,9,B1,17.0,TO,1702.0,Norte,1718204.0,Porto Nacional,77500***,PF,***.829.741-**,***,GD.TO.001.780.880,2023-03-30,R,Caracterizada como Autoconsumo remoto,2,UFV,Radia�ão solar,Microgeracao,600,NaN,NaN,NaN,NaN,NaN
1311301,2023-07-31,07/2023,8.467115e+12,CEEE-D,COMPANHIA ESTADUAL DE DISTRIBUI�ÃO DE ENERGIA ...,1,Residencial,9,B1,43.0,RS,4306.0,Sul,4301602.0,Bagé,96425***,PF,***.578.200-**,***,GD.RS.001.503.106,2022-04-20,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,436,NaN,NaN,NaN,NaN,NaN
768985,2023-07-31,07/2023,3.467321e+12,EMT,Energisa Mato Grosso - Distribuidora de Energi...,1,Residencial,9,B1,51.0,MT,5101.0,Centro Oeste,5108006.0,Tapurah,78573***,PF,***.699.492-**,***,GD.MT.000.830.904,2022-01-19,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,600,NaN,NaN,NaN,NaN,NaN
45423,2023-07-31,07/2023,8.324196e+12,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,1,Residencial,9,B1,24.0,RN,2404.0,Nordeste,2408102.0,Natal,59134***,PF,***.265.024-**,***,GD.RN.001.616.042,2023-01-16,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
674914,2023-07-31,07/2023,2.016440e+12,RGE SUL,RGE SUL DISTRIBUIDORA DE ENERGIA S.A.,1,Residencial,9,B1,43.0,RS,4301.0,Sul,4307005.0,Erechim,99714***,PF,***.821.530-**,***,GD.RS.001.019.788,2022-04-19,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,270,NaN,NaN,NaN,NaN,NaN
899035,2023-07-31,07/2023,1.543032e+12,Equatorial GO,EQUATORIAL GOIÁS DISTRIBUIDORA DE ENERGIA S/A,2,Comercial,11,B3,52.0,GO,5203.0,Centro Oeste,5208707.0,Goiânia,74393***,PF,***.379.021-**,***,GD.GO.001.310.972,2022-09-06,R,Caracterizada como Autoconsumo remoto,3,UFV,Radia�ão solar,Microgeracao,2000,NaN,NaN,NaN,NaN,NaN
264181,2023-07-31,07/2023,6.272793e+12,Equatorial MA,EQUATORIAL MARANHÃO DISTRIBUIDORA DE ENERGIA S.A,1,Residencial,9,B1,21.0,MA,2102.0,Nordeste,2107001.0,Montes Altos,65936***,PF,***.966.083-**,***,GD.MA.001.871.192,2023-05-31,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,276,NaN,NaN,NaN,NaN,NaN
1558793,2023-07-31,07/2023,4.368898e+12,COPEL-DIS,COPEL DISTRIBUI�ÃO S.A.,1,Residencial,9,B1,41.0,PR,4106.0,Sul,4114609.0,Marechal Cândido Rondon,85960***,PF,***.798.749-**,***,GD.PR.001.334.664,2022-05-18,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,1200,NaN,NaN,NaN,NaN,NaN
1852689,2023-07-31,07/2023,7.047251e+12,ENEL CE,COMPANHIA ENERGÉTICA DO CEARÁ,3,Rural,10,B2,23.0,CE,2303.0,Nordeste,2309607.0,Pacajus,62870***,PF,***.359.943-**,***,GD.CE.000.853.047,2022-02-25,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,300,NaN,NaN,NaN,NaN,NaN
263019,2023-07-31,07/2023,6.272793e+12,Equatorial MA,EQUATORIAL MARANHÃO DISTRIBUIDORA DE ENERGIA S.A,1,Residencial,9,B1,21.0,MA,2102.0,Nordeste,2109908.0,Santa Inês,65300***,PF,***.846.513-**,***,GD.MA.001.721.998,2023-03-21,P,Com Microgeracao ou Minigeracao distribuida,1,UFV,Radia�ão solar,Microgeracao,3300,NaN,NaN,NaN,NaN,NaN


In [15]:
# Tamanho total de (linhas , colunas)
df.shape

(2053221, 33)

In [16]:
# Todos os tipos de dados presentes
df.dtypes

DatGeracaoConjuntoDados          object
AnmPeriodoReferencia             object
NumCNPJDistribuidora            float64
SigAgente                        object
NomAgente                        object
CodClasseConsumo                  int64
DscClasseConsumo                 object
CodSubGrupoTarifario              int64
DscSubGrupoTarifario             object
codUFibge                       float64
SigUF                            object
codRegiao                       float64
NomRegiao                        object
CodMunicipioIbge                float64
NomMunicipio                     object
CodCEP                           object
SigTipoConsumidor                object
NumCPFCNPJ                       object
NomeTitularEmpreendimento        object
CodEmpreendimento                object
DthAtualizaCadastralEmpreend     object
SigModalidadeEmpreendimento      object
DscModalidadeHabilitado          object
QtdUCRecebeCredito                int64
SigTipoGeracao                   object


In [17]:
# Contagem de observações para cada coluna
df.count()

DatGeracaoConjuntoDados         2053221
AnmPeriodoReferencia            2053221
NumCNPJDistribuidora            2053081
SigAgente                       2053081
NomAgente                       2053081
CodClasseConsumo                2053221
DscClasseConsumo                2053221
CodSubGrupoTarifario            2053221
DscSubGrupoTarifario            2053221
codUFibge                       2049092
SigUF                           2053041
codRegiao                       2049092
NomRegiao                       2053221
CodMunicipioIbge                2053041
NomMunicipio                    2053041
CodCEP                          2053219
SigTipoConsumidor               2053219
NumCPFCNPJ                      2053219
NomeTitularEmpreendimento       2053212
CodEmpreendimento               2053220
DthAtualizaCadastralEmpreend    2053221
SigModalidadeEmpreendimento     2053221
DscModalidadeHabilitado         2053040
QtdUCRecebeCredito              2053221
SigTipoGeracao                  2053221


In [18]:
# Informações detalhadas
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2053221 entries, 0 to 2053220
Data columns (total 33 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   DatGeracaoConjuntoDados       object 
 1   AnmPeriodoReferencia          object 
 2   NumCNPJDistribuidora          float64
 3   SigAgente                     object 
 4   NomAgente                     object 
 5   CodClasseConsumo              int64  
 6   DscClasseConsumo              object 
 7   CodSubGrupoTarifario          int64  
 8   DscSubGrupoTarifario          object 
 9   codUFibge                     float64
 10  SigUF                         object 
 11  codRegiao                     float64
 12  NomRegiao                     object 
 13  CodMunicipioIbge              float64
 14  NomMunicipio                  object 
 15  CodCEP                        object 
 16  SigTipoConsumidor             object 
 17  NumCPFCNPJ                    object 
 18  NomeTitularEmpreendime

In [19]:
# Backup local
dfbackup1 = df.copy()

## Transformation
Aqui os dados extraídos precisam ser transformados em um formato adequado para análise, e para isso deve seguir alguns passos, que podem precisar serem repetidos ao longo do processo de análise.

### Limpeza de Dados
Remover dados duplicados, corrigir erros de digitação, tratar dados inconsistentes etc.

### Subsituições

#### Funções

Funções utilizadas nesta etapa

##### Função regex_replace

In [20]:
# função para substituição de caractere
def replace_single_char(text, target_char, replacement_char):
  '''Essa função retornará caractere substituído'''
  return re.sub(target_char, replacement_char, text)

##### Função exclusão NAN

In [21]:
def fnan(coluna, coluna_parametro):
  '''
  coluna = str
  filtro = str
  dicionario = {}
  coluna_parametro = str
  '''

  parametro = coluna_parametro

  lista = df[coluna].unique().tolist()
  if 'NAN' in lista:
    lista.remove('NAN')

  filtro = df.loc[df[coluna] == 'NAN']

  dicionario = {}

  for a, b in enumerate(lista):
    x = df[coluna_parametro][df[coluna]== lista[a]].unique().tolist()
    dicionario[str(x[0])] = float(b)

  for n in filtro[coluna_parametro]:
    for i, v in dicionario.items():
      if n == i:
        df[coluna] = df[coluna].replace('NAN', v)

  return df[coluna].unique()

#### Substituição de caracter especial

In [22]:
# definição dos caracteres
target_char = '�'
replacement_char = 'c'

# chamando a função
df = df.applymap(lambda x: replace_single_char(str(x), target_char, replacement_char))

#### Upper

In [23]:
# padronizando registros em uppercase
df = df.applymap(lambda x: x.upper() if isinstance(x, str) else x)

#### Backup

In [24]:
dfbackup2 = df.copy()

#### Substituição de caracter com expressão regular

In [25]:
#df = df[~df.apply(lambda row: any('*' in str(cell) for cell in row), axis=1)]

target_char = '*'
replacement_char = '0'

df = df.applymap(lambda x: replace_single_char(str(x), re.escape(target_char), replacement_char))

In [26]:
dfbackup3 = df.copy()

#### Verificando nulos e duplicados

In [27]:
# Verificar se há valores duplicados
df.duplicated().sum()

0

In [28]:
# Verificar se há valores nulos/ausentes
df.isnull().sum()

DatGeracaoConjuntoDados         0
AnmPeriodoReferencia            0
NumCNPJDistribuidora            0
SigAgente                       0
NomAgente                       0
CodClasseConsumo                0
DscClasseConsumo                0
CodSubGrupoTarifario            0
DscSubGrupoTarifario            0
codUFibge                       0
SigUF                           0
codRegiao                       0
NomRegiao                       0
CodMunicipioIbge                0
NomMunicipio                    0
CodCEP                          0
SigTipoConsumidor               0
NumCPFCNPJ                      0
NomeTitularEmpreendimento       0
CodEmpreendimento               0
DthAtualizaCadastralEmpreend    0
SigModalidadeEmpreendimento     0
DscModalidadeHabilitado         0
QtdUCRecebeCredito              0
SigTipoGeracao                  0
DscFonteGeracao                 0
DscPorte                        0
MdaPotenciaInstaladaKW          0
NumCoordNEmpreendimento         0
NumCoordEEmpre

In [29]:
# Verificar se há valores nulos/ausentes
df.isna().sum()

DatGeracaoConjuntoDados         0
AnmPeriodoReferencia            0
NumCNPJDistribuidora            0
SigAgente                       0
NomAgente                       0
CodClasseConsumo                0
DscClasseConsumo                0
CodSubGrupoTarifario            0
DscSubGrupoTarifario            0
codUFibge                       0
SigUF                           0
codRegiao                       0
NomRegiao                       0
CodMunicipioIbge                0
NomMunicipio                    0
CodCEP                          0
SigTipoConsumidor               0
NumCPFCNPJ                      0
NomeTitularEmpreendimento       0
CodEmpreendimento               0
DthAtualizaCadastralEmpreend    0
SigModalidadeEmpreendimento     0
DscModalidadeHabilitado         0
QtdUCRecebeCredito              0
SigTipoGeracao                  0
DscFonteGeracao                 0
DscPorte                        0
MdaPotenciaInstaladaKW          0
NumCoordNEmpreendimento         0
NumCoordEEmpre

In [30]:
# Fazendo um relatório para mostrar os valores únicos de cada atributo.
# O for vai garantir que a atividade seja feita automaticamentepara todos os atributos

for coluna in df:
    valor = (df[coluna].unique())
    print("-"*70)
    msg = f"Valores únicos para o atributo {coluna}:"
    print(f"{msg:^70}")
    print("-"*70)
    print(f"\n{valor}\n")
    print("-"*70)


----------------------------------------------------------------------
       Valores únicos para o atributo DatGeracaoConjuntoDados:        
----------------------------------------------------------------------

['2023-07-31']

----------------------------------------------------------------------
----------------------------------------------------------------------
         Valores únicos para o atributo AnmPeriodoReferencia:         
----------------------------------------------------------------------

['07/2023']

----------------------------------------------------------------------
----------------------------------------------------------------------
         Valores únicos para o atributo NumCNPJDistribuidora:         
----------------------------------------------------------------------

['8336783000190.0' '61695227000193.0' '2302100000106.0' '2328280000197.0'
 '33050196000188.0' '53859112000169.0' '4172213000151.0' '7282377000120.0'
 '60444437000146.0' '33050071000158.0'

In [31]:
# Verificando os valores presentes na coluna para identificar inconsistencias
for coluna in df:
    valor = (df[coluna].groupby(df[coluna]).count())
    print("-"*70)
    msg = f"Contando valores {coluna}:"
    print(f"{msg:^70}")
    print("-"*70)
    print(f"\n{valor}\n")
    print("-"*70)

----------------------------------------------------------------------
              Contando valores DatGeracaoConjuntoDados:               
----------------------------------------------------------------------

DatGeracaoConjuntoDados
2023-07-31    2053221
Name: DatGeracaoConjuntoDados, dtype: int64

----------------------------------------------------------------------
----------------------------------------------------------------------
                Contando valores AnmPeriodoReferencia:                
----------------------------------------------------------------------

AnmPeriodoReferencia
07/2023    2053221
Name: AnmPeriodoReferencia, dtype: int64

----------------------------------------------------------------------
----------------------------------------------------------------------
                Contando valores NumCNPJDistribuidora:                
----------------------------------------------------------------------

NumCNPJDistribuidora
10532365000110.0      

In [32]:
# Verificando valores únicos para codUFibge, inclusive NAN
df.codUFibge.unique()

array(['42.0', '35.0', '33.0', '32.0', '31.0', '29.0', 'NAN', '28.0',
       '26.0', '27.0', '25.0', '24.0', '23.0', '22.0', '21.0', '15.0',
       '16.0', '13.0', '14.0', '12.0', '53.0', '52.0', '11.0', '17.0',
       '51.0', '50.0', '41.0', '43.0'], dtype=object)

In [33]:
# Verificando valores únicos para codRegiao, inclusive NAN
df.codRegiao.unique()

array(['4204.0', '3515.0', '3513.0', '3507.0', '3506.0', '3505.0',
       '3502.0', '3501.0', '3503.0', '3504.0', '3508.0', '3512.0',
       '3511.0', '3510.0', '3306.0', '3301.0', '3302.0', '3304.0',
       '3203.0', '3204.0', '3201.0', '3202.0', '3107.0', '3108.0',
       '3109.0', '3112.0', '3110.0', '3105.0', '3102.0', '3103.0',
       '3104.0', '2905.0', '2903.0', '2907.0', '2906.0', 'NAN', '2902.0',
       '2803.0', '2802.0', '2801.0', '2605.0', '2603.0', '2602.0',
       '2703.0', '2504.0', '2503.0', '2502.0', '2501.0', '2404.0',
       '2402.0', '2401.0', '2303.0', '2301.0', '2302.0', '2305.0',
       '2307.0', '2306.0', '2304.0', '2202.0', '2203.0', '2101.0',
       '2104.0', '2105.0', '2102.0', '2103.0', '1503.0', '1505.0',
       '1506.0', '1602.0', '1303.0', '1401.0', '1402.0', '1202.0',
       '5301.0', '5204.0', '5203.0', '5205.0', '5201.0', '1101.0',
       '1102.0', '1702.0', '1701.0', '5104.0', '5103.0', '5101.0',
       '5102.0', '5105.0', '5002.0', '5004.0', '5003.0'

In [34]:
# Verificando valores únicos para CodMunicipioIbge, inclusive NAN
df.CodMunicipioIbge.unique()

array(['4210852.0', '3550308.0', '3510609.0', ..., '1304104.0',
       '1301506.0', '1301605.0'], dtype=object)

In [35]:
# Lista IBGE
#listaIbge = df.codUFibge.unique().tolist()
#if 'NAN' in listaIbge:
#  listaIbge.remove('NAN')

#listaIbge

In [36]:
# Verificando valores únicos para SigUF, inclusive NAN
df.SigUF.unique()

array(['SC', 'SP', 'RJ', 'ES', 'MG', 'BA', 'SE', 'PE', 'AL', 'PB', 'RN',
       'CE', 'PI', 'MA', 'PA', 'AP', 'AM', 'RR', 'AC', 'DF', 'GO', 'RO',
       'TO', 'MT', 'MS', 'PR', 'RS', 'NAN'], dtype=object)

In [37]:
# Lista UF únicas
listaUF = df.SigUF.unique()
listaUF = listaUF[:-1].tolist()
listaUF

['SC',
 'SP',
 'RJ',
 'ES',
 'MG',
 'BA',
 'SE',
 'PE',
 'AL',
 'PB',
 'RN',
 'CE',
 'PI',
 'MA',
 'PA',
 'AP',
 'AM',
 'RR',
 'AC',
 'DF',
 'GO',
 'RO',
 'TO',
 'MT',
 'MS',
 'PR',
 'RS']

In [38]:
dfbackup4 = df.copy()

#### Coluna UF

Ajusta coluna UF retirando valores 'NAN'

In [39]:
# filtro NAN para UF
filtroUF = df.loc[df['SigUF'] == 'NAN']

In [40]:
# Laço de repetição para substituição dos valores NAN em SigUF pela UF inserida no CodEmpreendimento
for n in filtroUF['CodEmpreendimento']:
  for m in listaUF:
    if str(n).startswith(f'GD.{m}'):
      df['SigUF'] = df['SigUF'].replace('NAN', f'{m}')
    else:
      pass

In [41]:
# Verificando valores únicos para SigUF, retirando NAN
df.SigUF.unique()

array(['SC', 'SP', 'RJ', 'ES', 'MG', 'BA', 'SE', 'PE', 'AL', 'PB', 'RN',
       'CE', 'PI', 'MA', 'PA', 'AP', 'AM', 'RR', 'AC', 'DF', 'GO', 'RO',
       'TO', 'MT', 'MS', 'PR', 'RS'], dtype=object)

In [42]:
# contagem número de agentes
df['SigUF'].value_counts()

SP    330705
RS    260536
MG    252624
PR    174880
BA    109749
RJ     98543
MT     91661
GO     87774
SC     80947
MS     77050
CE     65545
PE     62719
PA     58325
RN     47085
ES     39860
PI     38516
MA     38293
TO     29093
PB     25058
AL     20960
RO     18319
DF     16579
SE     10633
AM      8072
AC      4875
AP      2822
RR      1998
Name: SigUF, dtype: int64

#### Colunas COD IBGE

Ajuste das colunas a seguir, substituindo valores 'NAN'
* codRegiao
* codUFibge
* CodMunicipioIbge

In [43]:
# Criação filtro codUFibge
#filtroIbge = df.loc[df['codUFibge'] == 'NAN']

In [44]:
# Dicionário codUFibge
#dictIbge = {}

#for a, b in enumerate(listaIbge):
#  x = df.SigUF[df['codUFibge']== listaIbge[a]].unique().tolist()
#  dictIbge[str(x[0])] = float(b)

#dictIbge

In [45]:
# Iterando codUFibge conforme SigUF
#for j in filtroIbge['SigUF']:
#  for e, f in dictIbge.items():
#    if j == e:
#      df['codUFibge'] = df['codUFibge'].replace('NAN', f)

In [46]:
# contagem número de codUFibge
#df['codUFibge'].value_counts()

In [47]:
fnan('codRegiao', 'NomRegiao')

array(['4204.0', '3515.0', '3513.0', '3507.0', '3506.0', '3505.0',
       '3502.0', '3501.0', '3503.0', '3504.0', '3508.0', '3512.0',
       '3511.0', '3510.0', '3306.0', '3301.0', '3302.0', '3304.0',
       '3203.0', '3204.0', '3201.0', '3202.0', '3107.0', '3108.0',
       '3109.0', '3112.0', '3110.0', '3105.0', '3102.0', '3103.0',
       '3104.0', '2905.0', '2903.0', '2907.0', '2906.0', 2204.0, '2902.0',
       '2803.0', '2802.0', '2801.0', '2605.0', '2603.0', '2602.0',
       '2703.0', '2504.0', '2503.0', '2502.0', '2501.0', '2404.0',
       '2402.0', '2401.0', '2303.0', '2301.0', '2302.0', '2305.0',
       '2307.0', '2306.0', '2304.0', '2202.0', '2203.0', '2101.0',
       '2104.0', '2105.0', '2102.0', '2103.0', '1503.0', '1505.0',
       '1506.0', '1602.0', '1303.0', '1401.0', '1402.0', '1202.0',
       '5301.0', '5204.0', '5203.0', '5205.0', '5201.0', '1101.0',
       '1102.0', '1702.0', '1701.0', '5104.0', '5103.0', '5101.0',
       '5102.0', '5105.0', '5002.0', '5004.0', '5003.0

In [48]:
fnan('codUFibge', 'SigUF')

array(['42.0', '35.0', '33.0', '32.0', '31.0', '29.0', 29.0, '28.0',
       '26.0', '27.0', '25.0', '24.0', '23.0', '22.0', '21.0', '15.0',
       '16.0', '13.0', '14.0', '12.0', '53.0', '52.0', '11.0', '17.0',
       '51.0', '50.0', '41.0', '43.0'], dtype=object)

In [49]:
fnan('CodMunicipioIbge', 'NomMunicipio')

array(['4210852.0', '3550308.0', '3510609.0', ..., '1304104.0',
       '1301506.0', '1301605.0'], dtype=object)

#### Alterando numéricos NAN

In [79]:
# Verificando os valores presentes na coluna para identificar inconsistencias
for coluna in df:
    if df[coluna].dtype == 'float64' or df[coluna].dtype == 'int64':
      df[coluna] = df[coluna].fillna(0)
    elif coluna == 'CodMunicipioIbge':
      df[coluna] = df[coluna].fillna(0)
    else:
      pass

### Normalização de Dados
Colocar os dados em um formato padronizado para facilitar a análise.

Tipagem

In [52]:
df.sample(3)

,DatGeracaoConjuntoDados,AnmPeriodoReferencia,NumCNPJDistribuidora,SigAgente,NomAgente,CodClasseConsumo,DscClasseConsumo,CodSubGrupoTarifario,DscSubGrupoTarifario,codUFibge,SigUF,codRegiao,NomRegiao,CodMunicipioIbge,NomMunicipio,CodCEP,SigTipoConsumidor,NumCPFCNPJ,NomeTitularEmpreendimento,CodEmpreendimento,DthAtualizaCadastralEmpreend,SigModalidadeEmpreendimento,DscModalidadeHabilitado,QtdUCRecebeCredito,SigTipoGeracao,DscFonteGeracao,DscPorte,MdaPotenciaInstaladaKW,NumCoordNEmpreendimento,NumCoordEEmpreendimento,NomSubEstacao,NumCoordESub,NumCoordNSub
330958,2023-07-31,07/2023,19527639000158.0,EMR,ENERGISA MINAS RIO - DISTRIBUIDORA DE ENERGIA ...,1,RESIDENCIAL,9,B1,31.0,MG,3112.0,SUDESTE,3167608.0,SIMONÉSIA,36930000,PF,000.157.756-00,000,GD.MG.000.480.584,2021-01-30,R,CARACTERIZADA COMO AUTOCONSUMO REMOTO,2,UFV,RADIACÃO SOLAR,MICROGERACAO,500,NAN,NAN,NAN,NAN,NAN
878987,2023-07-31,07/2023,2341467000120.0,AME,AMAZONAS ENERGIA S.A.,1,RESIDENCIAL,9,B1,13.0,AM,1303.0,NORTE,1302603.0,MANAUS,69000000,PF,000.919.982-00,000,GD.AM.000.698.069,2021-11-18,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,1000,NAN,NAN,NAN,NAN,NAN
940114,2023-07-31,07/2023,90660754000160.0,COPREL,COPREL COOPERATIVA DE ENERGIA,2,COMERCIAL,11,B3,43.0,RS,4301.0,SUL,4320909.0,TAPEJARA,99950000,PJ,03951921000128,GAVIOLI & GAVIOLI - COMERCIO E REPRESENTACÕES ...,GD.RS.000.040.178,2018-09-27,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,500,NAN,NAN,NAN,NAN,NAN


In [80]:
# Realizando a tipagem das variaveis

df.CodClasseConsumo = df.CodClasseConsumo.astype(int)
df.CodSubGrupoTarifario = df.CodSubGrupoTarifario.astype(int)
df.codUFibge = df.codUFibge.astype(float).astype(int)
df.codRegiao = df.codRegiao.astype(float).astype(int)
df.CodMunicipioIbge = df.CodMunicipioIbge.astype(float).astype(int)
df.DthAtualizaCadastralEmpreend = df.DthAtualizaCadastralEmpreend.astype('datetime64[ns]')
df.QtdUCRecebeCredito = df.QtdUCRecebeCredito.astype(int)
df.MdaPotenciaInstaladaKW = df.MdaPotenciaInstaladaKW.astype(int)
df.NumCoordNEmpreendimento = df.NumCoordNEmpreendimento.astype(float)
df.NumCoordESub = df.NumCoordESub.astype(float)
df.NumCoordNSub = df.NumCoordNSub.astype(float)

AttributeError: ignored

In [68]:
df['anoCadastral'] = df.DthAtualizaCadastralEmpreend.dt.year

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2053221 entries, 0 to 2053220
Data columns (total 34 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   DatGeracaoConjuntoDados       object        
 1   AnmPeriodoReferencia          object        
 2   NumCNPJDistribuidora          object        
 3   SigAgente                     object        
 4   NomAgente                     object        
 5   CodClasseConsumo              int64         
 6   DscClasseConsumo              object        
 7   CodSubGrupoTarifario          int64         
 8   DscSubGrupoTarifario          object        
 9   codUFibge                     int64         
 10  SigUF                         object        
 11  codRegiao                     int64         
 12  NomRegiao                     object        
 13  CodMunicipioIbge              float64       
 14  NomMunicipio                  object        
 15  CodCEP                        ob

Tradução | Ajustes nomes

In [70]:
colunas_atuais = df.columns
colunas_atuais

Index(['DatGeracaoConjuntoDados', 'AnmPeriodoReferencia',
       'NumCNPJDistribuidora', 'SigAgente', 'NomAgente', 'CodClasseConsumo',
       'DscClasseConsumo', 'CodSubGrupoTarifario', 'DscSubGrupoTarifario',
       'codUFibge', 'SigUF', 'codRegiao', 'NomRegiao', 'CodMunicipioIbge',
       'NomMunicipio', 'CodCEP', 'SigTipoConsumidor', 'NumCPFCNPJ',
       'NomeTitularEmpreendimento', 'CodEmpreendimento',
       'DthAtualizaCadastralEmpreend', 'SigModalidadeEmpreendimento',
       'DscModalidadeHabilitado', 'QtdUCRecebeCredito', 'SigTipoGeracao',
       'DscFonteGeracao', 'DscPorte', 'MdaPotenciaInstaladaKW',
       'NumCoordNEmpreendimento', 'NumCoordEEmpreendimento', 'NomSubEstacao',
       'NumCoordESub', 'NumCoordNSub', 'anoCadastral'],
      dtype='object')

In [71]:
subs_colunas = ['dataPesquisa', 'periodoPesquisa',
       'idDistribuidora', 'agenteAbreviado', 'agenteNome', 'classeConsumo',
       'nomeClasseConsumo', 'subgrupoTarifa', 'nomeSubgrupoTarifa',
       'ufIbge', 'uf', 'regiaoIbge', 'regiaoNome', 'municipioIbge',
       'municipio', 'CEP', 'tipoConsumidor', 'idConsumidor',
       'nomeTitularEmpreendimento', 'idEmpreendimento',
       'cadastroEmpreendimento', 'tipoEmpreendimento',
       'tipoHabilitado', 'creditoReceber', 'tipoGeracao',
       'fonteGeracao', 'porte', 'potenciaInstalada(KW)',
       'coordNEmp', 'coordEEmp', 'subEstacao',
       'coordESub', 'coordNSub', 'anoCadastral']

In [72]:
df.columns = subs_colunas

In [73]:
df.columns

Index(['dataPesquisa', 'periodoPesquisa', 'idDistribuidora', 'agenteAbreviado',
       'agenteNome', 'classeConsumo', 'nomeClasseConsumo', 'subgrupoTarifa',
       'nomeSubgrupoTarifa', 'ufIbge', 'uf', 'regiaoIbge', 'regiaoNome',
       'municipioIbge', 'municipio', 'CEP', 'tipoConsumidor', 'idConsumidor',
       'nomeTitularEmpreendimento', 'idEmpreendimento',
       'cadastroEmpreendimento', 'tipoEmpreendimento', 'tipoHabilitado',
       'creditoReceber', 'tipoGeracao', 'fonteGeracao', 'porte',
       'potenciaInstalada(KW)', 'coordNEmp', 'coordEEmp', 'subEstacao',
       'coordESub', 'coordNSub', 'anoCadastral'],
      dtype='object')

In [74]:
df.dtypes

dataPesquisa                         object
periodoPesquisa                      object
idDistribuidora                      object
agenteAbreviado                      object
agenteNome                           object
classeConsumo                         int64
nomeClasseConsumo                    object
subgrupoTarifa                        int64
nomeSubgrupoTarifa                   object
ufIbge                                int64
uf                                   object
regiaoIbge                            int64
regiaoNome                           object
municipioIbge                       float64
municipio                            object
CEP                                  object
tipoConsumidor                       object
idConsumidor                         object
nomeTitularEmpreendimento            object
idEmpreendimento                     object
cadastroEmpreendimento       datetime64[ns]
tipoEmpreendimento                   object
tipoHabilitado                  

# **Qualidade e integridade dos dados**

### **Validação da estrutura dos dados (schema)**

In [81]:
# Definindo o esquema de validação e o examinando
schema = pa.DataFrameSchema({
    'dataPesquisa': pa.Column(pa.String),
    'periodoPesquisa': pa.Column(pa.String),
    'idDistribuidora': pa.Column(pa.String),
    'agenteAbreviado': pa.Column(pa.String),
    'agenteNome': pa.Column(pa.String),
    'classeConsumo': pa.Column(pa.Int),
    'nomeClasseConsumo': pa.Column(pa.String),
    'subgrupoTarifa': pa.Column(pa.Int),
    'nomeSubgrupoTarifa': pa.Column(pa.String),
    'ufIbge': pa.Column(pa.Int),
    'uf': pa.Column(pa.String),
    'regiaoIbge': pa.Column(pa.Int),
    'regiaoNome': pa.Column(pa.String),
    'municipioIbge': pa.Column(pa.Int),
    'municipio': pa.Column(pa.String),
    'CEP': pa.Column(pa.String),
    'tipoConsumidor': pa.Column(pa.String),
    'idConsumidor': pa.Column(pa.String),
    'nomeTitularEmpreendimento': pa.Column(pa.String),
    'idEmpreendimento': pa.Column(pa.String),
    'cadastroEmpreendimento': pa.Column(pa.Timestamp),
    'tipoEmpreendimento': pa.Column(pa.String),
    'tipoHabilitado': pa.Column(pa.String),
    'creditoReceber': pa.Column(pa.Int),
    'tipoGeracao': pa.Column(pa.String),
    'fonteGeracao': pa.Column(pa.String),
    'porte': pa.Column(pa.String),
    'potenciaInstalada(KW)': pa.Column(pa.Int),
    'coordNEmp': pa.Column(pa.Float),
    'coordEEmp': pa.Column(pa.String),
    'subEstacao': pa.Column(pa.String),
    'coordESub': pa.Column(pa.Float),
    'coordNSub': pa.Column(pa.Float),
    'anoCadastral': pa.Column(pa.Int)
})

schema.validate(df)

,dataPesquisa,periodoPesquisa,idDistribuidora,agenteAbreviado,agenteNome,classeConsumo,nomeClasseConsumo,subgrupoTarifa,nomeSubgrupoTarifa,ufIbge,uf,regiaoIbge,regiaoNome,municipioIbge,municipio,CEP,tipoConsumidor,idConsumidor,nomeTitularEmpreendimento,idEmpreendimento,cadastroEmpreendimento,tipoEmpreendimento,tipoHabilitado,creditoReceber,tipoGeracao,fonteGeracao,porte,potenciaInstalada(KW),coordNEmp,coordEEmp,subEstacao,coordESub,coordNSub,anoCadastral
0,2023-07-31,07/2023,8336783000190.0,CELESC-DIS,CELESC DISTRIBUICÃO S.A,5,PODER PÚBLICO,11,B3,42,SC,4204,SUL,4210852.0,MIRIM DOCE,00000000,PJ,95952289000155,MIRIM DOCE CAMARA DE VEREADORES,GD.SC.000.188.684,2020-01-20,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,500,0.0,NAN,NAN,0.0,0.0,2020
1,2023-07-31,07/2023,61695227000193.0,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,6,SERVIÇO PÚBLICO,7,A4,35,SP,3515,SUDESTE,3550308.0,SÃO PAULO,01109010,PJ,10882594001722,"INSTITUTO FEDERAL DE EDUCACÃO, CIÊNCIA E TECN.",GD.SP.000.027.612,2017-07-12,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,6996,0.0,NAN,NAN,0.0,0.0,2017
2,2023-07-31,07/2023,61695227000193.0,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,PODER PÚBLICO,9,B1,35,SP,3515,SUDESTE,3550308.0,SÃO PAULO,04115000,PF,000.015.418-00,000,GD.SP.000.081.118,2019-05-30,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,300,0.0,NAN,NAN,0.0,0.0,2019
3,2023-07-31,07/2023,61695227000193.0,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,PODER PÚBLICO,9,B1,35,SP,3515,SUDESTE,3510609.0,CARAPICUÍBA,06364000,PF,000.517.398-00,000,GD.SP.000.210.094,2020-03-30,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,300,0.0,NAN,NAN,0.0,0.0,2020
4,2023-07-31,07/2023,61695227000193.0,ELETROPAULO,ELETROPAULO METROPOLITANA ELETRICIDADE DE SÃO ...,5,PODER PÚBLICO,7,A4,35,SP,3515,SUDESTE,3505708.0,BARUERI,06462080,PJ,46523015000135,MUNICIPIO DE BARUERI,GD.SP.000.863.525,2021-04-02,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,2300,-2351.0,-4688.0,NAN,0.0,0.0,2021
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2053216,2023-07-31,07/2023,8324196000181.0,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,ILUMINACÃO PÚBLICA,9,B1,24,RN,2404,NORDESTE,2403251.0,PARNAMIRIM,59152000,PF,000.368.374-00,000,GD.RN.000.549.804,2021-07-31,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,737,0.0,NAN,NAN,0.0,0.0,2021
2053217,2023-07-31,07/2023,8324196000181.0,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,ILUMINACÃO PÚBLICA,9,B1,24,RN,2402,NORDESTE,2402006.0,CAICÓ,59300000,PF,000.405.104-00,000,GD.RN.000.863.246,2022-02-25,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,300,0.0,NAN,NAN,0.0,0.0,2022
2053218,2023-07-31,07/2023,8324196000181.0,COSERN,COMPANHIA ENERGÉTICA DO RIO GRANDE DO NORTE CO...,7,ILUMINACÃO PÚBLICA,10,B2,24,RN,2402,NORDESTE,2407203.0,MACAU,59500000,PF,000.771.634-00,000,GD.RN.000.349.903,2020-11-06,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,600,0.0,NAN,NAN,0.0,0.0,2020
2053219,2023-07-31,07/2023,7047251000170.0,ENEL CE,COMPANHIA ENERGÉTICA DO CEARÁ,7,ILUMINACÃO PÚBLICA,11,B3,23,CE,2303,NORDESTE,2304400.0,FORTALEZA,60878085,PJ,23552466000103,RENOVADORA DE P SAO LUIS LTDA,GD.CE.000.166.397,2019-12-31,P,COM MICROGERACAO OU MINIGERACAO DISTRIBUIDA,1,UFV,RADIACÃO SOLAR,MICROGERACAO,4460,0.0,NAN,NAN,0.0,0.0,2019


## **Colunas Persistidas**

=============================================================
STRING
=============================================================


* dataPesquisa                         
* periodoPesquisa                      
* idDistribuidora                      
* agenteAbreviado                      
* agenteNome                           
* nomeSubgrupoTarifa                   
* nomeClasseConsumo                    
* uf                                   
* regiaoNome                           
* municipio                            
* CEP                                  
* tipoConsumidor                       
* idConsumidor                         
* nomeTitularEmpreendimento            
* idEmpreendimento                     
* tipoEmpreendimento                   
* tipoHabilitado                       
* tipoGeracao                          
* fonteGeracao                         
* porte                                
* coordEEmp                            
* subEstacao                           

=============================================================
INT
=============================================================

* classeConsumo                         
* subgrupoTarifa                        
* anoCadastral                          
* creditoReceber                        
* potenciaInstalada(KW)                 

=============================================================
FLOAT
=============================================================

* coordNEmp                           
* coordESub                           
* coordNSub                           
* ufIbge                              
* regiaoSigla                         
* municipioIbge                       

=============================================================
DATETIME
=============================================================
* cadastroEmpreendimento       

## Copia de segurança do tratamento - Backup

In [82]:
# Backup para analise
df_tratado = df.copy()

## Load
Depois que os dados são extraídos e transformados adequadamente, eles estarão prontos para as análise, mas antes disso eles precisam ser carregados em um local de armazenamento adequado. Podendo ser um banco de dados SQL ou NoSQL, um sistema de armazenamento em nuvem, e para o caso de disponibilizar o projeto publicamente é ideal que ele seja colocar em uma pasta de datasets, diferenciando o arquivo bruto e o tratado.

In [84]:
# Carregametno GCP
df_tratado.to_csv('gs://projeto-final-ad2-e8/dados/tratados/tratado_geracao_distribuida.csv', index=False)

In [85]:
print('Notebook executado com SUCESSO!!!')

Notebook executado com SUCESSO!!!
